In [9]:
import numpy as np
import time
from robot.vrep_robot import VrepRobot
try:
    from robot.evolved_robot import EvolvedRobot
except ImportError as error:
    print(error.__class__.__name__ + ": " + 'DBus works only on linux!')
import vrep.vrep as vrep
from settings import Settings
from vision.tracker import Tracker, get_marker_object
from utility.path_tracking import transform_pos_angle
import math
from vrep.control_env import get_object_handle, get_pose, set_pose

In [2]:
vision_thread = Tracker(mid=5,
                        transform=None,
                        mid_aux=0,
                        video_source=-1,
                        capture=False,
                        show=True,
                        debug=False,
                       )

starting Tracker, video source:  -1


In [3]:
vision_thread.start()

Last markers len: 8
5Restore Calibration


In [4]:
thymio = {
    'name': 'thymio',
    'body': 'Thymio',
    'left_motor': 'leftMotor',
    'right_motor': 'rightMotor',
    'sensor': 'Proximity_sensor',
    'num_sensors': 7
}

settings = Settings(thymio)

In [11]:
# Initialize Vrep
vrep.simxFinish(-1)
clientID = vrep.simxStart(
    '127.0.0.1',
    19997,
    True,
    True,
    5000,
    5)

if clientID == -1:
    print('Failed connecting to remote API server')
    print('Program ended')

In [ ]:
# Initialize the robot
OP_MODE = vrep.simx_opmode_oneshot_wait
# robot = EvolvedRobot('thymio-II',clientID, None, OP_MODE, None, settings.robot_type)
robot = VrepRobot(clientID, None, OP_MODE, settings.robot_type)

In [ ]:
# Start simulation
if (vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking) == -1):
    print('Failed to start the simulation\n')
    print('Program ended\n')

In [117]:
# Get Robot and Set position & orientation
obstacle = get_marker_object(9)
print(obstacle.realxy())
robot_m = get_marker_object(7)
print(robot_m.realxy())
print(robot_m.orientation())
robot_current_position = robot_m.realxy()[:2]
angle = robot_m.orientation()
print(angle)
if angle >= 0.0 and angle <= math.pi/2:
    theta = (math.pi/2) - angle
else:
    theta = 2*np.pi - angle
    theta += np.pi/2
print(theta)
# update position and orientation of the robot in vrep
position, orientation = transform_pos_angle(
    robot_current_position, theta)
robot.v_set_pos_angle(position, [ 0, 0, theta ])

[0.63861422 0.59880258 0.        ]
[1.10135518 0.10174679 0.        ]
6.186267679385224
6.186267679385224
1.6677139545892592


In [92]:
# Get the position of all the obstacles in reality
obstacle = get_marker_object(9).realxy()
obstacle_0 = get_marker_object(10).realxy()
obstacle_1 = get_marker_object(11).realxy()
obstacles = [obstacle, obstacle_0, obstacle_1]
print(obstacle, '\n', obstacle_0, '\n', obstacle_1)

[0.63951524 0.59879241 0.        ] 
 [0.89368885 0.1130568  0.        ] 
 [0.16326747 0.43708016 0.        ]


In [66]:
# Get the position of all the walls in reality
marker_1 = get_marker_object(1).realxy()
marker_2 = get_marker_object(2).realxy()
marker_3 = get_marker_object(3).realxy()
marker_4 = get_marker_object(4).realxy()
markes = [marker_1, marker_2, marker_3, marker_4]
print(marker_1, '\n', marker_2, '\n', marker_3, '\n', marker_4)

[0.00359462 0.00074655 0.        ] 
 [ 1.19283681 -0.00128828  0.        ] 
 [1.19183735 0.79915987 0.        ] 
 [0.0024577  0.79757848 0.        ]


In [93]:
# obstacles handler vrep
obstacle_vrep = get_object_handle(clientID, 'obstacle')
obstacle_1_vrep = get_object_handle(clientID, 'obstacle1')
obstacle_2_vrep = get_object_handle(clientID, 'obstacle0')
obstacles_vrep = [obstacle_vrep, obstacle_1_vrep, obstacle_2_vrep]

Get object handle:  74
Get object handle:  77
Get object handle:  76


In [72]:
# wall handlers vrep
walls_vrep = [get_object_handle(clientID, wall) for wall in ('Wall', 'Wall0', 'Wall2', 'Wall3')]

Get object handle:  31
Get object handle:  32
Get object handle:  33
Get object handle:  34


In [40]:
set_pose(clientID, obstacle0_obj, [obstacle0[0], obstacle0[1], 0.099999])
# set_pose(clientID, obstacle1_obj, [obstacle1[0], obstacle1[1], 0.0999999])

SetPose object: 76  position:  [0.15145569136453008, 0.4298182869815179, 0.099999]
SetPose object: 77  position:  [0.8862664893474349, 0.11410803770585867, 0.0999999]


[0.8862664893474349, 0.11410803770585867, 0.0999999]

In [57]:
wall2 = get_object_handle(clientID, 'Wall2')
get_pose(clientID, wall2)

Get object handle:  33
Get Pose object: 33  position:  [0.574999988079071, 0.020999999716877937, 0.09999997913837433]


[0.574999988079071, 0.020999999716877937, 0.09999997913837433]

In [94]:
for obs, handler in zip(obstacles, obstacles_vrep):
    set_pose(clientID, handler, [obs[0], obs[1], 0.099999])

SetPose object: 74  position:  [0.6395152434137974, 0.5987924057058053, 0.099999]
SetPose object: 77  position:  [0.893688847432547, 0.11305680021494423, 0.099999]
SetPose object: 76  position:  [0.16326747351962573, 0.4370801616628016, 0.099999]
